In [1]:
import pandas as pd
import numpy as np

In [2]:
from sktime.performance_metrics.forecasting import median_absolute_percentage_error, mean_absolute_scaled_error, MeanSquaredScaledError


In [5]:
df_pred = pd.read_csv('../data/04_cv/results_cv_pred_FiveCitiesPMData_Chengdu_0_NP_horizon_[1].csv')
train_folds = pd.read_csv('../data/04_train_folds/train_fold_FiveCitiesPMData_Chengdu_0_NP_horizon_[1].csv')

In [6]:
train_folds

,ds,y,fold,tr/vl,config.n_lags,config.n_forecasts,config.learning_rate,config.epochs,config.num_hidden_layers,config.d_hidden,config.time_series,config.method,config.freq,config.test_size,config.val_size,config.lr,config.n_epoch
0,2013-01-01 00:00:00,121.0,0,tr,12,1,0.001,100,0,16,FiveCitiesPMData_Chengdu_0,NP,H,6570,1314,0.001,100
1,2013-01-01 01:00:00,134.0,0,tr,12,1,0.001,100,0,16,FiveCitiesPMData_Chengdu_0,NP,H,6570,1314,0.001,100
2,2013-01-01 02:00:00,168.5,0,tr,12,1,0.001,100,0,16,FiveCitiesPMData_Chengdu_0,NP,H,6570,1314,0.001,100
3,2013-01-01 03:00:00,203.0,0,tr,12,1,0.001,100,0,16,FiveCitiesPMData_Chengdu_0,NP,H,6570,1314,0.001,100
4,2013-01-01 04:00:00,217.0,0,tr,12,1,0.001,100,0,16,FiveCitiesPMData_Chengdu_0,NP,H,6570,1314,0.001,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769225,2015-04-02 01:00:00,22.0,4,vl,12,1,0.100,100,4,16,FiveCitiesPMData_Chengdu_0,NP,H,6570,1314,0.100,100
769226,2015-04-02 02:00:00,21.0,4,vl,12,1,0.100,100,4,16,FiveCitiesPMData_Chengdu_0,NP,H,6570,1314,0.100,100
769227,2015-04-02 03:00:00,22.0,4,vl,12,1,0.100,100,4,16,FiveCitiesPMData_Chengdu_0,NP,H,6570,1314,0.100,100
769228,2015-04-02 04:00:00,26.0,4,vl,12,1,0.100,100,4,16,FiveCitiesPMData_Chengdu_0,NP,H,6570,1314,0.100,100


In [7]:
df_pred.columns

Index(['step', 'true_value_1', 'fold', 'forecast_1', 'config.n_lags',
       'config.n_forecasts', 'config.learning_rate', 'config.epochs',
       'config.num_hidden_layers', 'config.d_hidden', 'config.time_series',
       'config.method', 'config.freq', 'config.test_size', 'config.val_size',
       'config.lr', 'config.n_epoch', 'config.training_time',
       'config.predicting_time'],
      dtype='object')

In [8]:
cols_config = [col for col in df_pred.columns if 'config.' in col]

In [9]:
cols_config

['config.n_lags',
 'config.n_forecasts',
 'config.learning_rate',
 'config.epochs',
 'config.num_hidden_layers',
 'config.d_hidden',
 'config.time_series',
 'config.method',
 'config.freq',
 'config.test_size',
 'config.val_size',
 'config.lr',
 'config.n_epoch',
 'config.training_time',
 'config.predicting_time']

In [10]:
ts = df_pred['config.time_series'].unique()
cols_config.remove('config.time_series')

In [11]:
ts

array(['FiveCitiesPMData_Chengdu_0'], dtype=object)

In [12]:
method = df_pred['config.method'].unique()[0]
cols_config.remove('config.method')

In [13]:
method

'NP'

In [14]:
cols_config.remove('config.training_time')
cols_config.remove('config.predicting_time')
cols_config.remove('config.freq')
cols_config.remove('config.test_size')

In [15]:
cols_config

['config.n_lags',
 'config.n_forecasts',
 'config.learning_rate',
 'config.epochs',
 'config.num_hidden_layers',
 'config.d_hidden',
 'config.val_size',
 'config.lr',
 'config.n_epoch']

In [16]:
configurations = df_pred[cols_config].drop_duplicates().reset_index(drop = True)

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sktime.performance_metrics.forecasting import median_absolute_percentage_error, mean_absolute_scaled_error

In [18]:
metrices = []
for time_series in ts:
    df_ts = df_pred[df_pred['config.time_series'] == time_series]
    train_ts = train_folds[train_folds['config.time_series'] == time_series]
    metrices_ts = []
    for row_n, config in configurations.iterrows():
        print(config.to_dict())
        metrics_df = pd.DataFrame()
        n_forecasts = config.to_dict()['config.n_forecasts']
        part = pd.DataFrame(config.to_dict(), index = [0]).merge(df_ts, on = cols_config, how = 'left')
        part_train = pd.DataFrame(config.to_dict(), index = [0]).merge(train_ts, on = cols_config, how = 'left')
        folds = part['fold'].unique()
        metrics_df['fold'] = folds
        metrics_df['ts'] = time_series
        metrics_df['method'] = method
        for col in cols_config:
            metrics_df[col] = config.to_dict()[col]
        for fold in folds:
            part_fold = part[part['fold'] == fold]
            part_train_fold = part_train[part_train['fold'] == fold]
            part_train_fold = part_train_fold[part_train_fold['tr/vl'] == 'tr']
            y_train = part_train_fold.y.values
            for frcst in range(1, n_forecasts+1):
                
                y_true = part_fold[[f'true_value_{frcst}']].values
                y_pred = part_fold[[f'forecast_{frcst}']].values
                metrics_df.loc[metrics_df['fold'] == fold,
                               f'MSE_forecast_{frcst}'] = mean_squared_error(y_true, y_pred)
                metrics_df.loc[metrics_df['fold'] == fold,
                               f'RMSE_forecast_{frcst}'] = np.sqrt(mean_squared_error(y_true, y_pred))
                metrics_df.loc[metrics_df['fold'] == fold,
                               f'MAE_forecast_{frcst}'] = mean_absolute_error(y_true, y_pred)
                metrics_df.loc[metrics_df['fold'] == fold,
                               f'MASE_forecast_{frcst}'] = mean_absolute_scaled_error(y_true = y_true,
                                                                                      y_pred = y_pred,
                                                                                      y_train = y_train)
                metrics_df.loc[metrics_df['fold'] == fold,
                               f'MAPE_forecast_{frcst}'] = median_absolute_percentage_error(y_true = y_true,
                                                                                      y_pred = y_pred)
                
                msse = MeanSquaredScaledError()
                metrics_df.loc[metrics_df['fold'] == fold,
                               f'MSSE_forecast_{frcst}'] = msse(y_true=y_true,
                                                                y_pred=y_pred,
                                                                y_train=y_train)
        metrices_ts.append(metrics_df)
    metrices.append(pd.concat(metrices_ts))


{'config.n_lags': 12, 'config.n_forecasts': 1, 'config.learning_rate': 0.001, 'config.epochs': 100, 'config.num_hidden_layers': 0, 'config.d_hidden': 16, 'config.val_size': 1314, 'config.lr': '0.001_0.001_0.001_0.001_0.001', 'config.n_epoch': '100_100_100_100_100'}


ValueError: You are trying to merge on object and float64 columns. If you wish to proceed you should use pd.concat

In [21]:
cols_config

['config.n_lags',
 'config.n_forecasts',
 'config.learning_rate',
 'config.epochs',
 'config.num_hidden_layers',
 'config.d_hidden',
 'config.val_size',
 'config.lr',
 'config.n_epoch']

In [26]:
pd.DataFrame(config.to_dict(), index = [0])

,config.n_lags,config.n_forecasts,config.learning_rate,config.epochs,config.num_hidden_layers,config.d_hidden,config.val_size,config.lr,config.n_epoch
0,12,1,0.001,100,0,16,1314,0.001_0.001_0.001_0.001_0.001,100_100_100_100_100


In [35]:
train_ts[cols_config].drop_duplicates().iloc[0].to_dict()

{'config.n_lags': 12.0,
 'config.n_forecasts': 1.0,
 'config.learning_rate': 0.001,
 'config.epochs': 100.0,
 'config.num_hidden_layers': 0.0,
 'config.d_hidden': 16.0,
 'config.val_size': 1314.0,
 'config.lr': 0.001,
 'config.n_epoch': 100.0}

In [27]:
ts = pd.read_pickle('../data/02_intermediate/FiveCitiesPMData_Chengdu_0.pickle')

In [28]:
ts

,ds,y
26304,2013-01-01 00:00:00,121.0
26305,2013-01-01 01:00:00,134.0
26306,2013-01-01 02:00:00,168.5
26307,2013-01-01 03:00:00,203.0
26308,2013-01-01 04:00:00,217.0
...,...,...
52579,2015-12-31 19:00:00,176.0
52580,2015-12-31 20:00:00,146.0
52581,2015-12-31 21:00:00,162.0
52582,2015-12-31 22:00:00,169.0


In [36]:
part_train = pd.DataFrame(train_ts[cols_config].drop_duplicates().iloc[0].to_dict(), index = [0]).merge(train_ts, on = cols_config, how = 'left')

In [37]:
part_train

,config.n_lags,config.n_forecasts,config.learning_rate,config.epochs,config.num_hidden_layers,config.d_hidden,config.val_size,config.lr,config.n_epoch,ds,y,fold,tr/vl,config.time_series,config.method,config.freq,config.test_size
0,12.0,1.0,0.001,100.0,0.0,16.0,1314.0,0.001,100.0,2013-01-01 00:00:00,121.0,0,tr,FiveCitiesPMData_Chengdu_0,NP,H,6570
1,12.0,1.0,0.001,100.0,0.0,16.0,1314.0,0.001,100.0,2013-01-01 01:00:00,134.0,0,tr,FiveCitiesPMData_Chengdu_0,NP,H,6570
2,12.0,1.0,0.001,100.0,0.0,16.0,1314.0,0.001,100.0,2013-01-01 02:00:00,168.5,0,tr,FiveCitiesPMData_Chengdu_0,NP,H,6570
3,12.0,1.0,0.001,100.0,0.0,16.0,1314.0,0.001,100.0,2013-01-01 03:00:00,203.0,0,tr,FiveCitiesPMData_Chengdu_0,NP,H,6570
4,12.0,1.0,0.001,100.0,0.0,16.0,1314.0,0.001,100.0,2013-01-01 04:00:00,217.0,0,tr,FiveCitiesPMData_Chengdu_0,NP,H,6570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85465,12.0,1.0,0.001,100.0,0.0,16.0,1314.0,0.001,100.0,2015-04-02 01:00:00,22.0,4,vl,FiveCitiesPMData_Chengdu_0,NP,H,6570
85466,12.0,1.0,0.001,100.0,0.0,16.0,1314.0,0.001,100.0,2015-04-02 02:00:00,21.0,4,vl,FiveCitiesPMData_Chengdu_0,NP,H,6570
85467,12.0,1.0,0.001,100.0,0.0,16.0,1314.0,0.001,100.0,2015-04-02 03:00:00,22.0,4,vl,FiveCitiesPMData_Chengdu_0,NP,H,6570
85468,12.0,1.0,0.001,100.0,0.0,16.0,1314.0,0.001,100.0,2015-04-02 04:00:00,26.0,4,vl,FiveCitiesPMData_Chengdu_0,NP,H,6570


In [38]:
ts

,ds,y
26304,2013-01-01 00:00:00,121.0
26305,2013-01-01 01:00:00,134.0
26306,2013-01-01 02:00:00,168.5
26307,2013-01-01 03:00:00,203.0
26308,2013-01-01 04:00:00,217.0
...,...,...
52579,2015-12-31 19:00:00,176.0
52580,2015-12-31 20:00:00,146.0
52581,2015-12-31 21:00:00,162.0
52582,2015-12-31 22:00:00,169.0


In [24]:
import sktime

In [25]:
sktime.__version__

'0.7.0'

In [23]:
pd.concat(metrices)

,fold,ts,method,config.random_state,config.n_lags,config.n_forecasts,MSE_forecast_1,RMSE_forecast_1,MAE_forecast_1,MASE_forecast_1,...,MAE_forecast_4,MASE_forecast_4,MAPE_forecast_4,MSSE_forecast_4,MSE_forecast_5,RMSE_forecast_5,MAE_forecast_5,MASE_forecast_5,MAPE_forecast_5,MSSE_forecast_5
0,0,AirQualityUCI_0,RF,42,12,1,6394.396853,79.964973,57.558214,0.776105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AirQualityUCI_0,RF,42,12,1,8011.571652,89.507383,67.827051,0.926754,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,AirQualityUCI_0,RF,42,12,1,5335.556864,73.044896,49.391547,0.680356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,AirQualityUCI_0,RF,42,12,1,6929.080765,83.241100,53.674140,0.748620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,AirQualityUCI_0,RF,42,12,1,6208.761335,78.795694,58.309440,0.824478,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,AirQualityUCI_0,RF,42,12,5,6432.193263,80.200955,57.667240,0.777575,...,139.589926,1.882206,0.107154,2.881659,41445.771250,203.582345,157.810271,2.127885,0.123999,3.652131
1,1,AirQualityUCI_0,RF,42,12,5,8075.302818,89.862689,68.225125,0.932193,...,145.050521,1.981897,0.097976,3.173981,42501.310529,206.158460,161.439502,2.205827,0.117836,3.836641
2,2,AirQualityUCI_0,RF,42,12,5,5375.659290,73.318888,49.602424,0.683261,...,107.320301,1.478311,0.078702,1.938879,25666.414126,160.207410,120.943780,1.665971,0.092107,2.342243
3,3,AirQualityUCI_0,RF,42,12,5,6970.844030,83.491581,53.801864,0.750402,...,111.673404,1.557565,0.077768,2.226972,27217.546711,164.977413,117.802395,1.643049,0.081103,2.529075
4,4,AirQualityUCI_0,RF,42,12,5,6259.941624,79.119793,58.708874,0.830126,...,115.688119,1.635795,0.087422,2.154759,24834.410258,157.589372,120.191971,1.699478,0.090847,2.348801


In [127]:
y_train

array([1360., 1292., 1402., ..., 1118., 1205., 1274.])

In [66]:
pd.DataFrame(config.to_dict(), index = [0]).merge(df_pred, on = cols_config, how = 'left')

,config.random_state,config.n_lags,config.n_forecasts,step,true_value_1,fold,forecast_1,config.time_series,config.method,config.freq,...,config.training_time,config.predicting_time,true_value_2,true_value_3,true_value_4,true_value_5,forecast_2,forecast_3,forecast_4,forecast_5
0,42,24,5,0,1306.0,0,1262.42,test_0,RF,H,...,0.07041406631469727_0.0717470645904541_0.07451...,0.01651787757873535_0.015007972717285156_0.015...,1326.0,1473.0,1609.0,1611.0,1229.46,1311.14,1334.39,1400.79
1,42,24,5,0,1621.0,1,1578.51,test_0,RF,H,...,0.07041406631469727_0.0717470645904541_0.07451...,0.01651787757873535_0.015007972717285156_0.015...,1444.0,1418.0,1534.0,1484.0,1546.94,1598.82,1513.82,1483.06
2,42,24,5,0,1367.0,2,1438.60,test_0,RF,H,...,0.07041406631469727_0.0717470645904541_0.07451...,0.01651787757873535_0.015007972717285156_0.015...,1344.0,1130.0,1062.0,1076.0,1431.44,1461.51,1447.62,1432.54
3,42,24,5,0,1028.0,3,1056.38,test_0,RF,H,...,0.07041406631469727_0.0717470645904541_0.07451...,0.01651787757873535_0.015007972717285156_0.015...,1155.0,1235.0,1332.0,1445.0,1264.58,1356.62,1426.85,1468.12
4,42,24,5,0,1416.0,4,1450.21,test_0,RF,H,...,0.07041406631469727_0.0717470645904541_0.07451...,0.01651787757873535_0.015007972717285156_0.015...,1281.0,1207.0,1258.0,1458.0,1477.26,1469.41,1416.17,1398.11
5,42,24,5,0,1306.0,0,1262.42,test_1,RF,H,...,0.06993913650512695_0.07074093818664551_0.0735...,0.016290903091430664_0.015146017074584961_0.01...,1326.0,1473.0,1609.0,1611.0,1229.46,1311.14,1334.39,1400.79
6,42,24,5,0,1621.0,1,1578.51,test_1,RF,H,...,0.06993913650512695_0.07074093818664551_0.0735...,0.016290903091430664_0.015146017074584961_0.01...,1444.0,1418.0,1534.0,1484.0,1546.94,1598.82,1513.82,1483.06
7,42,24,5,0,1367.0,2,1438.60,test_1,RF,H,...,0.06993913650512695_0.07074093818664551_0.0735...,0.016290903091430664_0.015146017074584961_0.01...,1344.0,1130.0,1062.0,1076.0,1431.44,1461.51,1447.62,1432.54
8,42,24,5,0,1028.0,3,1056.38,test_1,RF,H,...,0.06993913650512695_0.07074093818664551_0.0735...,0.016290903091430664_0.015146017074584961_0.01...,1155.0,1235.0,1332.0,1445.0,1264.58,1356.62,1426.85,1468.12
9,42,24,5,0,1416.0,4,1450.21,test_1,RF,H,...,0.06993913650512695_0.07074093818664551_0.0735...,0.016290903091430664_0.015146017074584961_0.01...,1281.0,1207.0,1258.0,1458.0,1477.26,1469.41,1416.17,1398.11


In [63]:
df_pred

,step,true_value_1,fold,forecast_1,config.random_state,config.n_lags,config.n_forecasts,config.time_series,config.method,config.freq,...,config.training_time,config.predicting_time,true_value_2,true_value_3,true_value_4,true_value_5,forecast_2,forecast_3,forecast_4,forecast_5
0,0,1306.0,0,1275.54,42,12,1,test_0,RF,H,...,0.06956601142883301_0.06760716438293457_0.0681...,0.003325223922729492_0.0030078887939453125_0.0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1326.0,0,1241.10,42,12,1,test_0,RF,H,...,0.06956601142883301_0.06760716438293457_0.0681...,0.003325223922729492_0.0030078887939453125_0.0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1473.0,0,1229.10,42,12,1,test_0,RF,H,...,0.06956601142883301_0.06760716438293457_0.0681...,0.003325223922729492_0.0030078887939453125_0.0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1609.0,0,1503.70,42,12,1,test_0,RF,H,...,0.06956601142883301_0.06760716438293457_0.0681...,0.003325223922729492_0.0030078887939453125_0.0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1611.0,0,1522.03,42,12,1,test_0,RF,H,...,0.06956601142883301_0.06760716438293457_0.0681...,0.003325223922729492_0.0030078887939453125_0.0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0,1306.0,0,1262.42,42,24,5,test_1,RF,H,...,0.06993913650512695_0.07074093818664551_0.0735...,0.016290903091430664_0.015146017074584961_0.01...,1326.0,1473.0,1609.0,1611.0,1229.46,1311.14,1334.39,1400.79
116,0,1621.0,1,1578.51,42,24,5,test_1,RF,H,...,0.06993913650512695_0.07074093818664551_0.0735...,0.016290903091430664_0.015146017074584961_0.01...,1444.0,1418.0,1534.0,1484.0,1546.94,1598.82,1513.82,1483.06
117,0,1367.0,2,1438.60,42,24,5,test_1,RF,H,...,0.06993913650512695_0.07074093818664551_0.0735...,0.016290903091430664_0.015146017074584961_0.01...,1344.0,1130.0,1062.0,1076.0,1431.44,1461.51,1447.62,1432.54
118,0,1028.0,3,1056.38,42,24,5,test_1,RF,H,...,0.06993913650512695_0.07074093818664551_0.0735...,0.016290903091430664_0.015146017074584961_0.01...,1155.0,1235.0,1332.0,1445.0,1264.58,1356.62,1426.85,1468.12
